In [42]:
import os

import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

from src.ingestion.report import Report
from src.llm_agents.kpi_simple_extraction.agent import KPISimpleExtractionAgent

from warnings import  simplefilter

simplefilter(action='ignore', category=Warning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
reports = []
list_paths = os.listdir("data")
for path in tqdm(list_paths):
    if path.endswith('.json'):
        report = Report.from_json("data/"+path)
        report.get_kpis()

  0%|          | 0/17 [00:00<?, ?it/s]

Parsing texts for  IBM   2023  ...
Loading embeddings for  IBM   2023  ...


  6%|▌         | 1/17 [00:48<13:01, 48.84s/it]

Parsing texts for  Siemens   2022  ...
Loading embeddings for  Siemens   2022  ...


 12%|█▏        | 2/17 [02:01<15:43, 62.90s/it]

...

 82%|████████▏ | 14/17 [11:09<02:33, 51.21s/it]

Parsing texts for  Siemens   2021  ...
Loading embeddings for  Siemens   2021  ...


 88%|████████▊ | 15/17 [12:16<01:51, 55.71s/it]

Parsing texts for  ABB   2021  ...
Loading embeddings for  ABB   2021  ...


100%|██████████| 17/17 [13:08<00:00, 46.41s/it]


In [27]:
df = pd.DataFrame(reports)
df = df.explode("questions").reset_index(drop=True)
df = df.assign(**df["questions"].apply(pd.Series))
df = df.assign(**df["response"].apply(pd.Series))
df = df.assign(**df["KPI"].apply(pd.Series))
df = df.explode("values")
df = df.query("not values.isna()").reset_index(drop=True)
df = df.assign(**df["values"].apply(pd.Series))
df.found = df.found.apply(lambda x: 1 if x == "True" or x else 0)
df.drop(columns=["questions", "response", "KPI", "values"], inplace=True)

In [34]:
df_sorted = df.sort_values(by=["file", "source"]).reset_index(drop=True)

In [36]:
df_sorted.to_json("data/kpi_extraction.json", orient="records", lines=True)

In [37]:
df_sorted

,file,question,acronym,full_name,description,found,formula,value,source
0,ABB_2021.json,What are the company's cash flow from operations?,CFO,Cash Flow from Operations,Cash flow from operations represents the amoun...,1,NaN,"3,338 million USD",48
1,ABB_2021.json,What is the company's total revenue?,Total Revenue,Total Revenue,The total amount of income generated by the co...,1,NaN,"28,945",139
2,ABB_2021.json,What is the company's gross margin (percentage...,Gross Margin,Gross Margin Percentage,Gross margin is a company's total sales revenu...,1,NaN,32.7%,140
3,ABB_2021.json,What are the company's current assets?,Current Assets,Current Assets,Current assets are all the assets of a company...,1,NaN,"18,695",155
4,ABB_2021.json,What are the company's current liabilities?,Current Liabilities,Current Liabilities,The company's obligations that are due within ...,1,NaN,"15,568",156
...,...,...,...,...,...,...,...,...,...
493,Siemens_2023.json,What is the company's gross profit?,Gross Profit,Gross Profit,Gross profit is the difference between revenue...,1,NaN,"29,653 million €",151
494,Siemens_2023.json,What is the company's net income?,Net Income,Net Income,Net income is the total profit of a company af...,1,NaN,"8,529",151
495,Siemens_2023.json,What are the company's cash flow from operations?,CFO,Cash Flow from Operations,Cash flow from operations represents the amoun...,1,NaN,"12,281 million €",151
496,Siemens_2023.json,What is the company's total revenue?,TR,Total Revenue,Total revenue represents the total amount of i...,1,NaN,"77,769 million €",155
